In [13]:
IDENTIFIER = "beethoven-op67"
NUM = 1

picklePath = f"../Pickles/{IDENTIFIER}/{IDENTIFIER}_"
numPath = f"../Pickles/{IDENTIFIER}/{NUM}/{IDENTIFIER}_{NUM}_"
dataPath = f"../Data/{IDENTIFIER}/{NUM}/{IDENTIFIER}_{NUM}_"

import pandas as pd
import json
import networkx as nx
import numpy as np
from dwave.system import DWaveSampler, FixedEmbeddingComposite

from qubo import createBQM
from samples import duplicates, overlaps, extractChosen


phrases = pd.read_csv(picklePath + "phrases.csv", index_col=[0,1])
G = nx.read_graphml(picklePath + "graph.graphml")
instruments = json.load(open(numPath + "instruments.json"))
embedding = json.load(open(numPath + "embedding.json", "r"))

sampler = FixedEmbeddingComposite(DWaveSampler(), embedding)

## Pegasus topology

- Each qubit coupled to 15 other qubits
- 5640 qubits in one QPU

Due to limitations with QPU size could only pick shortish pieces/cut pieces short.

In [14]:
print(sum(len(embedding[i]) for i in embedding))

4457


In [2]:
# Node constraint

multipliers = np.arange(1.5, 2, .5)
reads = 2000

file = pd.DataFrame()

for m in multipliers:
    bqm = createBQM(G, phrases, instruments, m, 1, 1)
    sampleset = sampler.sample(bqm, num_reads=reads, label=f"Haydn_{reads}")
    sample = sampleset.first

    new_row = pd.DataFrame({
        "Multiplier": [m],
        "Lowest energy": [bqm.energy(sample.sample)],
        "Chain break fraction": [sample.chain_break_fraction],
        "Duplicates": [duplicates(sample.sample, G)]
        })
    
    file = pd.concat([file, new_row])
    
    print(f"Multiplier {m} recorded!")

file.to_csv(dataPath+"lagrange-node.csv", index=False, mode="a", header=False)

Multiplier 1.5 recorded!


In [9]:
print(sample.sample)
print(sum(1 for n in range(4)))

{'Bassoon_10_Flute': 0, 'Bassoon_11_Flute': 0, 'Bassoon_12_Flute': 0, 'Bassoon_13_Flute': 0, 'Bassoon_14_Flute': 0, 'Bassoon_15_Flute': 0, 'Bassoon_16_Flute': 0, 'Bassoon_17_Flute': 0, 'Bassoon_18_Flute': 0, 'Bassoon_19_Flute': 0, 'Bassoon_1_Flute': 0, 'Bassoon_20_Flute': 0, 'Bassoon_21_Flute': 0, 'Bassoon_22_Flute': 0, 'Bassoon_23_Flute': 0, 'Bassoon_24_Flute': 0, 'Bassoon_25_Flute': 0, 'Bassoon_26_Flute': 0, 'Bassoon_27_Flute': 0, 'Bassoon_2_Flute': 1, 'Bassoon_3_Flute': 0, 'Bassoon_4_Flute': 1, 'Bassoon_5_Flute': 0, 'Bassoon_6_Flute': 0, 'Bassoon_7_Flute': 0, 'Bassoon_8_Flute': 1, 'Bassoon_9_Flute': 0, 'B♭ Clarinet_10_Flute': 0, 'B♭ Clarinet_11_Flute': 0, 'B♭ Clarinet_12_Flute': 0, 'B♭ Clarinet_13_Flute': 0, 'B♭ Clarinet_14_Flute': 0, 'B♭ Clarinet_15_Flute': 0, 'B♭ Clarinet_16_Flute': 0, 'B♭ Clarinet_17_Flute': 0, 'B♭ Clarinet_18_Flute': 0, 'B♭ Clarinet_19_Flute': 0, 'B♭ Clarinet_1_Flute': 0, 'B♭ Clarinet_20_Flute': 0, 'B♭ Clarinet_21_Flute': 0, 'B♭ Clarinet_22_Flute': 0, 'B♭ Clarin

In [ ]:
# Edge constraint

multipliers = np.arange(1.0, 10.5, .5)
reads = 2000

file = pd.DataFrame()

for m in multipliers:
    bqm = createBQM(G, phrases, instruments, m, 1, 1)
    sampleset = sampler.sample(bqm, num_reads=reads, label=f"Haydn_{reads}")
    sample = sampleset.first

    new_row = pd.DataFrame({
        "Multiplier": [m],
        "Lowest energy": [bqm.energy(sample.sample)],
        "Chain break fraction": [sample.chain_break_fraction],
        "Overlaps": [overlaps(G)]
        })
    
    file = pd.concat([file, new_row])
    
    print(f"Multiplier {m} recorded!")

file.to_csv(dataPath+"lagrange-node.csv", index=False, mode="a", header=True)